# Demo

This is a demonstration of the basic Python usage of the `anarci-toolz` package.

Please refer to the README for guidance on usage on the command line interface tool.

In [1]:
import pandas as pd
from anarci_toolz.pipeline import run_anarci_toolz

Fetch data from open source database of therapeutic antibodies from [TheraSabDab](https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/therasabdab/search/)

In [2]:
TARGET_URL = "https://opig.stats.ox.ac.uk/webapps/sabdab-sabpred/static/downloads/TheraSAbDab_SeqStruc_OnlineDownload.csv"
df = pd.read_csv(TARGET_URL)

In [3]:
# Do some light transformation to prepare data
df = df[df['Format']=="Whole mAb"]
n_sample = 100
df = df.sample(n_sample)
df = df[['Therapeutic', 'Format', 'HeavySequence', 'LightSequence']]

# Note: here we designate the amino acid sequence column as "sequence_aa"
df_long = pd.melt(df, id_vars=['Therapeutic', 'Format'], value_vars=['HeavySequence', 'LightSequence'], var_name='chain_type_input', value_name='sequence_aa')

In [4]:
df_long.head()

,Therapeutic,Format,chain_type_input,sequence_aa
0,Orticumab,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSNAWMSWVRQAPGKGLE...
1,Enokizumab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSYYWIEWVRQAPGQGLE...
2,Bimekizumab,Whole mAb,HeavySequence,EVQLVESGGGLVQPGGSLRLSCAASGFTFSDYNMAWVRQAPGKGLE...
3,Micvotabart,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSFSMSWVRQAPGKGLE...
4,Ozuriftamab,Whole mAb,HeavySequence,EVQLVQSGAEVKKPGESLRISCKGSGYTFTEYTMHWVRQAPGQGLE...


In [5]:
# Run anarci-toolz

df_result = run_anarci_toolz(
    df=df_long,
    scheme="imgt",
    allowed_species=["human"],
    seq_aa_header="sequence_aa",)

2025-02-21 14:35:40.973 | INFO     | anarci_toolz.abnumber_tool:run_parallel_abnumber:62 - Starting AbNumber processing ...
Processing sequences: 100%|██████████| 200/200 [00:00<00:00, 55093.97seq/s]
2025-02-21 14:35:41.661 | SUCCESS  | anarci_toolz.abnumber_tool:parallel_get_region_seqs:138 - AbNumber parallel processing complete
2025-02-21 14:35:41.729 | INFO     | anarci_toolz.anarci_tool:run_parallel_anarci:91 - Starting ANARCI processing ...
Processing sequences: 100%|██████████| 200/200 [00:00<00:00, 66900.14seq/s]
2025-02-21 14:35:42.262 | SUCCESS  | anarci_toolz.anarci_tool:run_parallel_anarci:104 - ANARCI parallel processing complete
2025-02-21 14:35:42.299 | SUCCESS  | anarci_toolz.pipeline:run_anarci_toolz:86 - anarci-toolz run complete.


We can see a comprehensive annotated ANARCI result from the `anarci-toolz`:

In [6]:
df_result.head()

,Therapeutic,Format,chain_type_input,sequence_aa,scheme,passed_abnumber,sequence_alignment_aa,species,chain_type,v_gene,...,fr1_aa,fr2_aa,fr3_aa,fr4_aa,passed_anarci,variable_region_start_index,variable_region_end_index,e_value,bitscore,bias
0,Orticumab,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSNAWMSWVRQAPGKGLE...,imgt,True,EVQLLESGGGLVQPGGSLRLSCAASGFTFSNAWMSWVRQAPGKGLE...,human,H,IGHV3-23*01,...,EVQLLESGGGLVQPGGSLRLSCAAS,MSWVRQAPGKGLEWVSS,YYADSVKGRSTISRDNSKNTLYLQMNSLRAEDTAVYYC,WGQGTLVTVSS,True,0,121,9.500000e-61,194.7,0.7
1,Enokizumab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSYYWIEWVRQAPGQGLE...,imgt,True,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSYYWIEWVRQAPGQGLE...,human,H,IGHV1-69*01,...,QVQLVQSGAEVKKPGSSVKVSCKAS,IEWVRQAPGQGLEWMGE,NPNEKFKGRVTITADESTSTAYMELSSLRSEDTAVYYC,WGQGTLVTVSS,True,0,122,2.300000e-56,180.6,1.6
2,Bimekizumab,Whole mAb,HeavySequence,EVQLVESGGGLVQPGGSLRLSCAASGFTFSDYNMAWVRQAPGKGLE...,imgt,True,EVQLVESGGGLVQPGGSLRLSCAASGFTFSDYNMAWVRQAPGKGLE...,human,H,IGHV3-7*01,...,EVQLVESGGGLVQPGGSLRLSCAAS,MAWVRQAPGKGLEWVAT,YYRDSVKGRFTISRDNAKNSLYLQMNSLRAEDTAVYYC,WGQGTLVTVSS,True,0,125,2.700000e-58,186.8,0.3
3,Micvotabart,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSFSMSWVRQAPGKGLE...,imgt,True,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSFSMSWVRQAPGKGLE...,human,H,IGHV3-23*01,...,EVQLLESGGGLVQPGGSLRLSCAAS,MSWVRQAPGKGLEWVSS,YYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYC,WGQGTLVTVSS,True,0,116,1.400000e-61,197.4,0.5
4,Ozuriftamab,Whole mAb,HeavySequence,EVQLVQSGAEVKKPGESLRISCKGSGYTFTEYTMHWVRQAPGQGLE...,imgt,True,EVQLVQSGAEVKKPGESLRISCKGSGYTFTEYTMHWVRQAPGQGLE...,human,H,IGHV5-10-1*01,...,EVQLVQSGAEVKKPGESLRISCKGS,MHWVRQAPGQGLEWMGG,GYNQKFKGRVTISADKSISTAYLQWSSLKASDTAMYYC,WGQGTLVTVSS,True,0,122,2.400000e-57,183.7,2.8


In [7]:
# Display residue view 

df_result = run_anarci_toolz(
    df=df_long,
    scheme="imgt",
    allowed_species=["human"],
    seq_aa_header="sequence_aa",
    display_residue_view=True)

2025-02-21 14:36:36.938 | INFO     | anarci_toolz.abnumber_tool:run_parallel_abnumber:62 - Starting AbNumber processing ...
Processing sequences: 100%|██████████| 200/200 [00:00<00:00, 80404.56seq/s]
2025-02-21 14:36:37.626 | SUCCESS  | anarci_toolz.abnumber_tool:parallel_get_region_seqs:138 - AbNumber parallel processing complete
2025-02-21 14:36:38.364 | INFO     | anarci_toolz.anarci_tool:run_parallel_anarci:91 - Starting ANARCI processing ...
Processing sequences: 100%|██████████| 200/200 [00:00<00:00, 54531.68seq/s]
2025-02-21 14:36:38.931 | SUCCESS  | anarci_toolz.anarci_tool:run_parallel_anarci:104 - ANARCI parallel processing complete
2025-02-21 14:36:38.979 | SUCCESS  | anarci_toolz.pipeline:run_anarci_toolz:86 - anarci-toolz run complete.


In [8]:
df_result.head()

,Therapeutic,Format,chain_type_input,sequence_aa,scheme,passed_abnumber,sequence_alignment_aa,species,chain_type,v_gene,...,imgt_pos_125,imgt_pos_126,imgt_pos_127,imgt_pos_128,passed_anarci,variable_region_start_index,variable_region_end_index,e_value,bitscore,bias
0,Orticumab,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSNAWMSWVRQAPGKGLE...,imgt,True,EVQLLESGGGLVQPGGSLRLSCAASGFTFSNAWMSWVRQAPGKGLE...,human,H,IGHV3-23*01,...,T,V,S,S,True,0,121,9.500000e-61,194.7,0.7
1,Enokizumab,Whole mAb,HeavySequence,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSYYWIEWVRQAPGQGLE...,imgt,True,QVQLVQSGAEVKKPGSSVKVSCKASGGTFSYYWIEWVRQAPGQGLE...,human,H,IGHV1-69*01,...,T,V,S,S,True,0,122,2.300000e-56,180.6,1.6
2,Bimekizumab,Whole mAb,HeavySequence,EVQLVESGGGLVQPGGSLRLSCAASGFTFSDYNMAWVRQAPGKGLE...,imgt,True,EVQLVESGGGLVQPGGSLRLSCAASGFTFSDYNMAWVRQAPGKGLE...,human,H,IGHV3-7*01,...,T,V,S,S,True,0,125,2.700000e-58,186.8,0.3
3,Micvotabart,Whole mAb,HeavySequence,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSFSMSWVRQAPGKGLE...,imgt,True,EVQLLESGGGLVQPGGSLRLSCAASGFTFSSFSMSWVRQAPGKGLE...,human,H,IGHV3-23*01,...,T,V,S,S,True,0,116,1.400000e-61,197.4,0.5
4,Ozuriftamab,Whole mAb,HeavySequence,EVQLVQSGAEVKKPGESLRISCKGSGYTFTEYTMHWVRQAPGQGLE...,imgt,True,EVQLVQSGAEVKKPGESLRISCKGSGYTFTEYTMHWVRQAPGQGLE...,human,H,IGHV5-10-1*01,...,T,V,S,S,True,0,122,2.400000e-57,183.7,2.8
